In [1]:
import pandas as pd
import numpy as np

In [2]:
first_names = pd.read_csv('firstnames.csv', sep=';')
first_names.head()

,name,gender,Great Britain,Ireland,U.S.A.,Italy,Malta,Portugal,Spain,France,...,"Kazakhstan/Uzbekistan,etc.",Turkey,Arabia/Persia,Israel,China,India/Sri Lanka,Japan,Korea,Vietnam,other countries
0,Aad,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aadam,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aadje,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aadu,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaf,?F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
name_gender = first_names[['name', 'gender']]

In [4]:
name_gender.head()

,name,gender
0,Aad,M
1,Aadam,M
2,Aadje,F
3,Aadu,M
4,Aaf,?F


In [5]:
name_gender = name_gender.assign(gender_cleaned=name_gender['gender'].apply(lambda g: g[1] if len(g) == 2 else g))
name_gender = name_gender[['name', 'gender_cleaned']].rename(columns={'gender_cleaned': 'gender'}).dropna()
name_gender = name_gender[name_gender['name'].str.len() > 3]
name_gender = name_gender[name_gender['gender'] != '?']

In [6]:
name_gender.head()

,name,gender
1,Aadam,M
2,Aadje,F
3,Aadu,M
5,Aafke,F
7,Aafkea,F


In [7]:
polydata = pd.read_csv('polynames.csv')
polydata.head()

,polyname
0,â™¥ ðŸ”½ðŸ”º ðŸ”½â™¥
1,Hertzi 2018
2,Sophie Will
3,Cyb3rPolyLELder
4,Sinaeder


In [8]:
polynames = {}
for polyname in polydata.polyname:
    names = []
    for name in name_gender.name.unique():
        if name != 'Eder' and name in polyname:
            names.append(name)
    names_cleaned = names.copy()
    for name1 in names:
        for name2 in names:
            if name1 in name2 and name1 != name2 and name1 in names_cleaned:
                names_cleaned.remove(name1)
    polynames[polyname] = names_cleaned
            

In [9]:
matches = []
for polyname, names in polynames.items():
    if not names:
        matches.append((polyname, '', 1))
    for name in names:
        matches.append((polyname, name, 1/len(names)))
matches = pd.DataFrame(matches, columns=['polyname', 'first_name', 'multiplicity'])

In [10]:
matches.head()

,polyname,first_name,multiplicity
0,â™¥ ðŸ”½ðŸ”º ðŸ”½â™¥,,1.0
1,Hertzi 2018,,1.0
2,Sophie Will,Sophie,0.5
3,Sophie Will,Will,0.5
4,Cyb3rPolyLELder,,1.0


In [11]:
polyname_gender = matches.join(name_gender.drop_duplicates().set_index('name'), on='first_name').fillna('N')
polyname_gender.head()

,polyname,first_name,multiplicity,gender
0,â™¥ ðŸ”½ðŸ”º ðŸ”½â™¥,,1.0,N
1,Hertzi 2018,,1.0,N
2,Sophie Will,Sophie,0.5,F
3,Sophie Will,Will,0.5,M
4,Cyb3rPolyLELder,,1.0,N


In [12]:
polyname_gender.groupby('gender').aggregate(lambda x: np.sum(x)/len(polyname_gender))

,multiplicity
gender,
F,0.193258
M,0.226183
N,0.540201
